#Harshvardhan Aditya

#E20CSE365

In [26]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from sklearn.utils import shuffle
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns




import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import numpy as np
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier 
from sklearn import metrics
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from sklearn.metrics import auc, plot_precision_recall_curve
import keras
from keras.datasets import mnist
from keras.models import Sequential,model_from_json
from keras.layers import Dense
from keras.optimizers import rmsprop_v2
import pylab as plt
from sklearn.utils.class_weight import compute_class_weight

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
data = pd.read_json("/content/drive/MyDrive/News_Category_Dataset_v2.json", lines=True)
data.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [29]:
data.isnull().sum().any()

False

In [30]:
data.drop(['authors','link','date'], inplace=True, axis=1)
data.head()

,category,headline,short_description
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,She left her husband. He killed their children...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Of course it has a song.
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,The actor and his longtime girlfriend Anna Ebe...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,The actor gives Dems an ass-kicking for not fi...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,"The ""Dietland"" actress said using the bags is ..."


In [31]:
categories = data.groupby('category')
print("total categories:", categories.ngroups)
print(categories.size())

total categories: 41
category
ARTS               1509
ARTS & CULTURE     1339
BLACK VOICES       4528
BUSINESS           5937
COLLEGE            1144
COMEDY             5175
CRIME              3405
CULTURE & ARTS     1030
DIVORCE            3426
EDUCATION          1004
ENTERTAINMENT     16058
ENVIRONMENT        1323
FIFTY              1401
FOOD & DRINK       6226
GOOD NEWS          1398
GREEN              2622
HEALTHY LIVING     6694
HOME & LIVING      4195
IMPACT             3459
LATINO VOICES      1129
MEDIA              2815
MONEY              1707
PARENTING          8677
PARENTS            3955
POLITICS          32739
QUEER VOICES       6314
RELIGION           2556
SCIENCE            2178
SPORTS             4884
STYLE              2254
STYLE & BEAUTY     9649
TASTE              2096
TECH               2082
THE WORLDPOST      3664
TRAVEL             9887
WEDDINGS           3651
WEIRD NEWS         2670
WELLNESS          17827
WOMEN              3490
WORLD NEWS         2177
WORLDPOST 

In [32]:
women = data[data['category']=='WOMEN'].index.values
qv = data[data['category']=='QUEER VOICES'].index.values
fifty = data[data['category']=='FIFTY'].index.values
hl = data[data['category']=='HOME & LIVING'].index.values
wn = data[data['category']=='WEIRD NEWS'].index.values
lv = data[data['category']=='LATINO VOICES'].index.values
impact = data[data['category']=='IMPACT'].index.values


data.drop(index = qv, inplace = True, axis=0)
data.drop(index = fifty, inplace = True, axis=0)
data.drop(index = hl, inplace = True, axis=0)
data.drop(index = wn, inplace = True, axis=0)
data.drop(index = lv, inplace = True, axis=0)
data.drop(index = impact, inplace = True, axis=0)
data.drop(index = women, inplace = True, axis=0)

In [33]:
data.category = data.category.map(lambda x: "ARTS , CULTURE & ENVIRONMENT" if x == "CULTURE & ARTS" or x =='ARTS' or  x == "GREEN" or  x == "ENVIRONMENT" or x == 'ARTS & CULTURE' else x)
data.category = data.category.map(lambda x: "BLACK LIVES MATTER" if x == "BLACK VOICES" else x)
data.category = data.category.map(lambda x: "BUSINESS & MONEY" if x == "BUSINESS" or x =='MONEY' else x)
data.category = data.category.map(lambda x: "LAW & CRIME" if x == "DIVORCE" or x =='CRIME' else x)
data.category = data.category.map(lambda x: "SPORTS & EDUCATION" if x == "EDUCATION" or x =='TECH' or x == 'SCIENCE' or x == 'SPORTS' or x == 'COLLEGE' else x)
data.category = data.category.map(lambda x: "ENTERTAINMENT" if x == "MEDIA" or x =='COMEDY' else x)
data.category = data.category.map(lambda x: "FOOD & TRAVEL" if x == "FOOD & DRINK" or x =='TRAVEL' or x =='TASTE' else x)
data.category = data.category.map(lambda x: "PARENTING" if x == "PARENTS"  else x)
data.category = data.category.map(lambda x: "STYLE & BEAUTY" if x == "STYLE"  else x)
data.category = data.category.map(lambda x: "WORLD NEWS" if x == "THE WORLDPOST" or x =='WORLDPOST' else x)
data.category = data.category.map(lambda x: "GOOD NEWS" if x == "WEDDINGS" or x =='HEALTHY LIVING' or x == 'WELLNESS' else x)
data.category = data.category.map(lambda x: "POLITICS & RELIGION" if x == "RELIGION" or x =='POLITICS' else x)

In [34]:
duplicate = data[data.duplicated(['short_description','headline','category'])].index.values
data.drop(index = duplicate, inplace = True, axis=0)

In [35]:
data.headline = data['headline'].str.lower()
data.short_description = data['short_description'].str.lower()

In [36]:
pr = data[data['category']=='POLITICS & RELIGION'].index.values
data.drop(index = pr, inplace = True, axis=0)

In [37]:
# finding & removing empty columns
print(len(data[data['headline'] == ""]))
data.loc[data['headline'] == "", 'headline'] = np.nan
data.dropna(subset=['headline'], inplace=True)
print(len(data[data['headline'] == ""]))

3
0


In [38]:
print(len(data[data['short_description'] == ""]))
data.loc[data['short_description'] == "", 'short_description'] = np.nan
data.dropna(subset=['short_description'], inplace=True)
print(len(data[data['short_description'] == ""]))

13345
0


In [39]:
# Displaying the categories after grouping
categories = data.groupby('category')
print("total categories:", categories.ngroups)
print(categories.size())

total categories: 11
category
ARTS , CULTURE & ENVIRONMENT     6583
BLACK LIVES MATTER               4119
BUSINESS & MONEY                 6781
ENTERTAINMENT                   20158
FOOD & TRAVEL                   17562
GOOD NEWS                       27774
LAW & CRIME                      6097
PARENTING                       12164
SPORTS & EDUCATION               9841
STYLE & BEAUTY                  11068
WORLD NEWS                       7080
dtype: int64


In [40]:
data['text'] = data['headline'] + " " + data['short_description']
data.drop(columns=['headline','short_description'], inplace=True, axis=1)

In [41]:
data.to_csv('cleaned_data.csv', index=False)

In [42]:
path = "cleaned_data.csv"
data = pd.read_csv(path)
print(data.head(3))

        category                                               text
0    LAW & CRIME  there were 2 mass shootings in texas last week...
1  ENTERTAINMENT  will smith joins diplo and nicky jam for the 2...
2  ENTERTAINMENT  hugh grant marries for the first time at age 5...


In [43]:
# Tokenizing
vocab_size = 10000
embedding_dim = 32
max_length = 150
trunc_type = 'post'
oov_tok = '<OOV>'
padding_post = 'post'

tokenizer = Tokenizer(num_words=vocab_size, oov_token = oov_tok, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X, padding=padding_post, maxlen=max_length, truncating=trunc_type)

In [44]:
keras.backend.clear_session()
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(keras.layers.Dense(11,activation='softmax'))

ann.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [45]:
Y = pd.get_dummies(data['category']).values

In [46]:
# Splitting data into train,valid,test
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
X_valid, X_test, Y_valid, Y_test = train_test_split(X_test,Y_test, test_size = 0.5, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_valid.shape,Y_valid.shape)
print(X_test.shape,Y_test.shape)

(103381, 150) (103381, 11)
(12923, 150) (12923, 11)
(12923, 150) (12923, 11)


In [47]:
ann.fit(X_train, Y_train, batch_size = 32, epochs = 10, validation_data=(X_valid,Y_valid))

Epoch 1/10
3231/3231 [==============================] - 10s 3ms/step - loss: 4.1229 - acc: 0.2060 - val_loss: 2.2405 - val_acc: 0.2109
Epoch 2/10
3231/3231 [==============================] - 5s 2ms/step - loss: 2.2411 - acc: 0.2154 - val_loss: 2.2402 - val_acc: 0.2109
Epoch 3/10
3231/3231 [==============================] - 5s 1ms/step - loss: 2.2410 - acc: 0.2154 - val_loss: 2.2410 - val_acc: 0.2109
Epoch 4/10
3231/3231 [==============================] - 5s 1ms/step - loss: 2.2410 - acc: 0.2154 - val_loss: 2.2411 - val_acc: 0.2109
Epoch 5/10
3231/3231 [==============================] - 5s 2ms/step - loss: 2.2408 - acc: 0.2154 - val_loss: 2.2411 - val_acc: 0.2109
Epoch 6/10
3231/3231 [==============================] - 5s 2ms/step - loss: 2.2409 - acc: 0.2154 - val_loss: 2.2412 - val_acc: 0.2109
Epoch 7/10
3231/3231 [==============================] - 5s 1ms/step - loss: 2.2409 - acc: 0.2154 - val_loss: 2.2413 - val_acc: 0.2109
Epoch 8/10
3231/3231 [==============================] - 5s 1m

In [48]:
Y_pred = ann.predict(X_test)
Y_pred = np.argmax(Y_pred, axis=1)
Y_test_ = np.argmax(Y_test, axis=1)

CM = confusion_matrix(Y_test_, Y_pred)
TP = CM[1][1]
FP = CM[0][1]

for _ in range(3):
  print()
print("-----------------------------------------------------------------------------")
print("Accuracy: ",metrics.accuracy_score(Y_test_, Y_pred))
print("TP Rate: ", TP)
print("FP Rate: ", FP)
print("MCC: ", metrics.matthews_corrcoef(Y_test_, Y_pred))
print("-----------------------------------------------------------------------------")
print(classification_report(Y_test_, Y_pred))




-----------------------------------------------------------------------------
Accuracy:  0.21512032809719106
TP Rate:  0
FP Rate:  0
MCC:  0.0
-----------------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       654
           1       0.00      0.00      0.00       391
           2       0.00      0.00      0.00       674
           3       0.00      0.00      0.00      2100
           4       0.00      0.00      0.00      1755
           5       0.22      1.00      0.35      2780
           6       0.00      0.00      0.00       601
           7       0.00      0.00      0.00      1186
           8       0.00      0.00      0.00       975
           9       0.00      0.00      0.00      1117
          10       0.00      0.00      0.00       690

    accuracy                           0.22     12923
   macro avg       0.02      0.09      0.03     12923
weighted avg      

In [49]:
cf_matrix = confusion_matrix(Y_test_, Y_pred)
Labels = ['ARTS , CULTURE & ENVIRONMENT', 'BLACK LIVES MATTER', 'BUSINESS & MONEY', 'ENTERTAINMENT', 'FOOD & TRAVEL', 'GOOD NEWS', 'LAW & CRIME', 'PARENTING', 'POLITICS & RELIGION', 'SPORTS & EDUCATION', 'STYLE & BEAUTY', 'WORLD NEWS']
plt.figure(figsize=(8, 8))
heatmap = sns.heatmap(cf_matrix, xticklabels=Labels, yticklabels=Labels, annot=True, fmt='d', color='blue')
plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.title('Confusion Matrix')
plt.show()